In [1]:
from games_setup import *
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule, MoleculeStoichiometry
from SBMLLint.common.reaction import Reaction
from SBMLLint.games.som import SOM
from SBMLLint.common.simple_sbml import SimpleSBML

import collections
import copy
import itertools
import networkx as nx
import numpy as np
import os
import pandas as pd
#
from SBMLLint.common.stoichiometry_matrix import StoichiometryMatrix
from SBMLLint.games.games_pp import GAMES_PP, SOMStoichiometry, SOMReaction, TOLERANCE
from SBMLLint.games.games_report import GAMESReport, SimplifiedReaction

import matplotlib.pyplot as plt
%matplotlib inline

ReactionOperation = collections.namedtuple("ReactionOperation", 
    "reaction operation")

Current Directory: /Users/woosubs/Desktop/ModelEngineering/SBMLLint/SBMLLint/notebooks


In [2]:
# table 1 - full table (res_list_full)
# statistics columns
MODEL_ID = "model_id"
NUM_TOTAL_REACTIONS = "num_total_reactions"
NUM_NONBDRY_REACTIONS = "num_nonbdry_reactions"
NUM_MULTIMULTI_REACTIONS = "num_multimulti_reactions"
#
ERROR_TYPE = "error_type"
TYPE_I = "type1"
TYPE_II = "type2"
TYPE_III = "type3"
CANCELING = "canceling"
ECHELON = "echelon"
ERRORS = [TYPE_I, TYPE_II, TYPE_III, CANCELING, ECHELON]
#
NUM_ERROR_REACTIONS = "num_error_reactions"

In [3]:
# table 2 - mean table (res_list_mean)
MODEL_ID = "model_id"
NUM_TOTAL_REACTIONS = "num_total_reactions"
NUM_NONBDRY_REACTIONS = "num_nonbdry_reactions"
NUM_MULTIMULTI_REACTIONS = "num_multimulti_reactions"
MEAN_ERROR_SIZE = "mean_error_size"
IS_GAMES_PLUS = "is_games_plus"

In [4]:
EXCEPTIONS = ["BIOMD0000000094.xml",
              "BIOMD0000000596.xml",
              "BIOMD0000000786.xml",
              "BIOMD0000000794.xml",
              "BIOMD0000000830.xml",
             ]
raw_files = [f for f in os.listdir(cn.BIOMODELS_DIR) if f[-4:] == ".xml"]
files = [f for f in raw_files if f not in EXCEPTIONS]
paths = [os.path.join(cn.BIOMODELS_DIR, filename) for filename in files]

In [5]:
len(files)

826

In [6]:
simple = SimpleSBML()
simple.initialize(os.path.join(cn.BIOMODELS_DIR, "BIOMD0000000167.xml"))
simple.reactions

[PstatDimerisation: 2.00 Pstat_sol -> PstatDimer_sol; (Kf_PstatDimerisation * pow(Pstat_sol, 2) + -(Kr_PstatDimerisation * PstatDimer_sol)) * sol,
 statDephosphorylation: Pstat_nuc -> stat_nuc; Kcat_dephos * statPhosphatase_nuc * Pstat_nuc * (1 / (Km_dephos + Pstat_nuc)) * nuc,
 statPhosphorylation: stat_sol -> Pstat_sol + species_test; Kcat_phos * statKinase_sol * stat_sol * (1 / (Km_phos + stat_sol)) * sol,
 PstatDimerisationNuc: 2.00 Pstat_nuc -> PstatDimer_nuc; (Kf_PstatDimerisation * pow(Pstat_nuc, 2) + -(Kr_PstatDimerisation * PstatDimer_nuc)) * nuc,
 PstatDimer__import: PstatDimer_sol -> PstatDimer_nuc; PstatDimer_impMax * PstatDimer_sol * (1 / (Kpsd_imp + PstatDimer_sol)) * nm,
 stat_export: stat_sol -> stat_nuc; nuc * stat_expMax * stat_nuc * (1 / (Ks_exp + stat_nuc)) * nm,
 stat_import: stat_sol -> stat_nuc; nuc * stat_impMax * stat_sol * (1 / (Ks_imp + stat_sol)) * nm]

In [7]:
m = GAMES_PP(simple)
res = m.analyze(simple_games=False)
print(m.error_summary)

Model analyzed...
At least one error found.

[ErrorSummary(type='type3', errors=[PstatDimerisation: 2.00 {Pstat_sol} -> 2.00 {Pstat_nuc=stat_nuc=stat_sol}]), ErrorSummary(type='echelon', errors=[statPhosphorylation:  -> {species_test}])]


In [8]:
gr = GAMESReport(m)
rep, rep_num = gr.reportEchelonError([m.echelon_errors[0]], explain_details=True)
print(rep)


We detected a mass imbalance
:  -> species_test

from the following reaction isolation set.

1. statPhosphorylation: stat_sol -> Pstat_sol + species_test
2. PstatDimerisation: 2.00 Pstat_sol -> PstatDimer_sol
3. PstatDimerisationNuc: 2.00 Pstat_nuc -> PstatDimer_nuc

----------------------------------------------------------------------
----------------------------------------------------------------------

These uni-uni reactions created mass-equivalence.
(The chemical species within a curly bracket have the same atomic mass.)

{PstatDimer_nuc=PstatDimer_sol} is inferred by:
4. PstatDimer__import: PstatDimer_sol -> PstatDimer_nuc

{Pstat_nuc=stat_nuc=stat_sol} is inferred by:
5. stat_export: stat_sol -> stat_nuc
6. statDephosphorylation: Pstat_nuc -> stat_nuc

----------------------------------------------------------------------

Based on the uni-uni reactions above, we create mass-equivalent pseudo reactions.

(pseudo 1.) statPhosphorylation: {Pstat_nuc=stat_nuc=stat_sol} -> {Pstat

In [9]:
rep_num

[3]

In [9]:
line1 = "\nWe detected a mass imbalance\n:  -> species_test\n\nfrom the following reaction isolation set.\n\n"
line2 = "1. statPhosphorylation: stat_sol -> Pstat_sol + species_test\n2. PstatDimerisation: 2.00 Pstat_sol -> PstatDimer_sol\n"
line3 = "3. PstatDimerisationNuc: 2.00 Pstat_nuc -> PstatDimer_nuc\n\n"
line4 = "----------------------------------------------------------------------\n----------------------------------------------------------------------\n\n"
line5 = "These uni-uni reactions created mass-equivalence.\n"
line6 = "(The chemical species within a curly bracket have the same atomic mass.)\n\n"
line7 = "{PstatDimer_nuc=PstatDimer_sol} is inferred by:\n4. PstatDimer__import: PstatDimer_sol -> PstatDimer_nuc\n\n"
line8 = "{Pstat_nuc=stat_nuc=stat_sol} is inferred by:\n5. statDephosphorylation: Pstat_nuc -> stat_nuc\n"
line9 = "6. stat_export: stat_sol -> stat_nuc\n\n----------------------------------------------------------------------\n\n"
line10 = "Based on the uni-uni reactions above, we create\nmass-equivalent pseudo reactions.\n\n"
line11 = "(pseudo 1.) statPhosphorylation:\n"
line11_1 = " "*10 + "{Pstat_nuc=stat_nuc=stat_sol} -> {Pstat_sol} + {species_test}\n"
line12 = "(pseudo 2.) PstatDimerisation:\n"
line12_1 = " "*20 + "2.00 {Pstat_sol} -> {PstatDimer_nuc=PstatDimer_sol}\n"
line13 = "(pseudo 3.) PstatDimerisationNuc:\n"
line13_1 = " "*2 + "2.00 {Pstat_nuc=stat_nuc=stat_sol} -> {PstatDimer_nuc=PstatDimer_sol}\n\n"
line14 = "----------------------------------------------------------------------\n\n"
line15 = "An operation between the pseudo reactions:\n1.00 * statPhosphorylation + 0.50 * PstatDimerisation -\n"
line15_1 = "0.50 * PstatDimerisationNuc\n\nwill result in empty reactant with zero mass:\n\n:  -> {species_test}\n\n\n"
line16 = "----------------------------------------------------------------------\n\n----------------------------------------------------------------------\n\n\n\n**********************************************************************\n\n"

In [10]:
print(line1 + line2 + line3 + line4 + line5 + line6 + 
     line7 + line8 + line9 + line10 + line11 + line11_1 +
     line12 + line12_1 + line13 + line13_1 + line14 + 
     line15 + line15_1 + line16)


We detected a mass imbalance
:  -> species_test

from the following reaction isolation set.

1. statPhosphorylation: stat_sol -> Pstat_sol + species_test
2. PstatDimerisation: 2.00 Pstat_sol -> PstatDimer_sol
3. PstatDimerisationNuc: 2.00 Pstat_nuc -> PstatDimer_nuc

----------------------------------------------------------------------
----------------------------------------------------------------------

These uni-uni reactions created mass-equivalence.
(The chemical species within a curly bracket have the same atomic mass.)

{PstatDimer_nuc=PstatDimer_sol} is inferred by:
4. PstatDimer__import: PstatDimer_sol -> PstatDimer_nuc

{Pstat_nuc=stat_nuc=stat_sol} is inferred by:
5. statDephosphorylation: Pstat_nuc -> stat_nuc
6. stat_export: stat_sol -> stat_nuc

----------------------------------------------------------------------

Based on the uni-uni reactions above, we create
mass-equivalent pseudo reactions.

(pseudo 1.) statPhosphorylation:
          {Pstat_nuc=stat_nuc=stat_sol}

In [11]:
m.echelon_errors

[statPhosphorylation:  -> {species_test}]